In [1]:
import pandas as pd
import numpy as np
from tflearn.data_utils import build_hdf5_image_dataset
import h5py
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import tflearn

The train and test dataset are already build

####Build the dataset (only required once)
id_train = pd.read_csv('../Data/id_train.csv')#replace by path of the id_train file 
file = pd.DataFrame(id_train['Id'].apply(lambda x: './train_imgs/' + str(x) + '.jpg' ))#Build the path to each image
file['label'] = id_train['label'] - 1 #Label goes from 1 to 4, we want 0 to 3 
file.to_csv('file_train.csv', sep = ' ', header = False, index=False)
dataset_file = 'file_train.csv'
build_hdf5_image_dataset(dataset_file, image_shape=(32, 32), mode='file', output_path='dataset.h5', categorical_labels=True, normalize=True)

Loading the Dataset

In [2]:
#Loading the dataset
h5f = h5py.File('train32_dataset.h5', 'r')
X  = h5f['X']
Y = h5f['Y']

In [3]:
# Shuffle the data
X, Y = shuffle(X, Y)

In [4]:
X.shape

(8000, 32, 32, 3)

In [5]:
Y.shape

(8000, 4)

In [6]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

In [7]:
# Create extra synthetic training data by flipping, rotating and blurring the images on our data set.
img_aug = ImageAugmentation()
#img_aug.add_random_flip_leftright() #Flipping will make 
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)

Building the network

In [8]:
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)

# Step 1: Convolution
network = conv_2d(network, 32, 3, activation='relu')

# Step 2: Max pooling
network = max_pool_2d(network, 4)

# Step 3: Convolution again
network = conv_2d(network, 64, 3, activation='relu')

# Step 4: Convolution yet again
network = conv_2d(network, 64, 3, activation='relu')

# Step 5: Max pooling again
network = max_pool_2d(network, 4)

# Step 6: Fully-connected 512 node neural network
network = fully_connected(network, 128, activation='relu')

# Step 7: Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

# Step 8: Fully-connected neural network with two outputs (0=isn't a bird, 1=is a bird) to make the final prediction
network = fully_connected(network, 4, activation='softmax')

In [9]:
# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy', n_classes = 4,learning_rate=0.01)

In [10]:
# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0)

Training the model

In [11]:
epochs = 1000

In [18]:
model.fit(X, Y, n_epoch=100, shuffle=True,
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
run_id='roof-classifier', validation_set=0.3)
model.save("roof-classifier-epoch1000-lr0.01.tfl")
print("Network trained and saved as roof-classifier.tfl!")

Training Step: 84080  | total loss: 0.77701
| Adam | epoch: 100 | loss: 0.77701 - acc: 0.6904 | val_loss: 0.70953 - val_acc: 0.7271 -- iter: 5600/5600
Training Step: 84080  | total loss: 0.77701
| Adam | epoch: 100 | loss: 0.77701 - acc: 0.6904 | val_loss: 0.70953 - val_acc: 0.7271 -- iter: 5600/5600
--
Network trained and saved as roof-classifier.tfl!


Prediction